##Installing Necessary Libraries
This cell installs or upgrades the necessary Python packages for the project:

* google-cloud-aiplatform: The Google Cloud AI Platform SDK, used for interacting with Vertex AI.
* requests: A popular HTTP library for making API calls (will be used for the Llumo API).
* nltk: Natural Language Toolkit, a leading platform for building Python programs to work with human language data.
* beautifulsoup4: A library for pulling data out of HTML and XML files, useful for web scraping.

The ! at the beginning allows running shell commands in Jupyter notebooks or Google Colab.

In [14]:
!pip install --upgrade google-cloud-aiplatform requests

##Setting Up VertexAI
This cell mounts your Google Drive to the Colab environment:

* It imports the `os` module for operating system operations and the drive module from `google.colab`.
* `drive.mount()` attaches your Google Drive to the '/content/drive' directory in the Colab environment.
* `force_remount=True` ensures that the drive is remounted even if it was previously mounted, which can help resolve connection issues

In [15]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


This cell sets up the Google Cloud credentials:

* It sets the `GOOGLE_APPLICATION_CREDENTIALS` environment variable to point to your Google Cloud service account key file.
* The file path suggests that your credentials JSON file is stored in your Google Drive under the 'MyDrive/vertex/' directory.
* This step is crucial for authenticating your script with Google Cloud services.

In [16]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/drive/MyDrive/googleCred.json'

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Importing Libraries
This cell imports all necessary Python modules:

* `os`: For operating system operations and environment variables.
* `requests`: For making HTTP requests (will be used for Llumo API calls).
* `json`: For JSON parsing and manipulation.
* `logging`: For setting up logging in the script.
* `getpass`: For securely inputting passwords or API keys.
* `aiplatform`: The main module for interacting with Vertex AI.
* `TextGenerationModel`: Specific class for text generation tasks in Vertex AI.



In [18]:
import os
import requests
import json
import logging
from getpass import getpass
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel

##Setting up Basic Logging
This cell sets up logging for the script:

* `logging.basicConfig()` configures the logging system with INFO level, meaning it will capture all info, warning, and error messages.
* `logger = logging.getLogger(__name__)` creates a logger object.` __name__` is a special Python variable that gets set to the module's name when the module is executed.

In [19]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

##Setting up Llumo API
This cell securely handles the Llumo API key:

* It uses `getpass()` to prompt for the Llumo API key without displaying it on the screen as it's typed.
* The API key is then stored as an environment variable named 'LLUMO_API_KEY'.
* This approach keeps the API key secure by not hardcoding it in the script.

In [20]:
llumo_api_key = getpass("Enter your Llumo API key: ")
os.environ['LLUMO_API_KEY'] = llumo_api_key
del llumo_api_key

Enter your Llumo API key: ··········


# Llumo Evaluation Function Documentation

## Define Llumo Evaluation Function

### Function Definition:
* We define a function `evaluate_with_llumo` that takes a `prompt` and `output` as inputs.

### API Setup:
* We retrieve the Llumo API key from environment variables.
* We set the API endpoint and prepare headers for the HTTP request.

### Payload Preparation:
* We create a payload dictionary with the `prompt`, `output`, and predefined analytics type ("Clarity").

### API Request:
* We use `requests.post()` to send a POST request to the Llumo API.
* `response.raise_for_status()` will raise an exception for HTTP errors.

### Response Parsing:
* We parse the JSON response and extract the evaluation data.
* We print the API response for debugging purposes.

### Error Handling:
* We use a try-except block to catch potential errors:
  * JSON decoding errors
  * Request exceptions

* If an error occurs, we log it and return an empty dictionary with a failure indicator.

### Return Values:
* The function returns a tuple containing:
  * Evaluation data (or empty dictionary if evaluation failed)
  * Success boolean

This function encapsulates the entire process of interacting with the Llumo API for text evaluation, including error handling and result processing.

In [21]:
def evaluate_with_llumo(prompt, output):
    LLUMO_API_KEY = os.getenv("LLUMO_API_KEY")
    LLUMO_ENDPOINT = "https://app.llumo.ai/api/create-eval-analytics"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {LLUMO_API_KEY}"
    }
    payload = {
        "prompt": prompt,
        "input": {},
        "output": output,
        "analytics": ["Clarity"]
    }

    try:
        response = requests.post(LLUMO_ENDPOINT, json=payload, headers=headers)

        try:
            result = response.json()
            print("API Response:", result)
            return result.get('data', {}), True
        except json.JSONDecodeError as json_error:
            logger.error(f"Error decoding JSON: {str(json_error)}")
            return {}, False

    except requests.RequestException as e:
       logger.error(f"Error evaluating with Llumo: {str(e)}")
       return {}, False

##Getting Respone from VertexAI.
###Defining the prompt:

* We create a detailed prompt about photosynthesis. This serves as our example text for compression.


###Sending request:

* We use the VertexAI client to send a request to the text-bison model.
* The messages parameter follows the chat format:

 * A system message sets the AI's role.
 * A user message contains our prompt.




###Displaying results:

* We print the AI's response to the prompt.
* Vertex AI doesn't provide any token usage. So, it's not possible to print token usage for checking.



In [32]:
prompt = """
Context: Flexicab: The Smart Rickshaw Management PlatformFlexicab is a revolutionary SaaS platform designed specifically for Rickshaw drivers and owners in bustling urban environments. It addresses the unique challenges faced by this vital mode of transportation, empowering them with modern tools to optimize their operations and increase their earnings.**Key Features:*** **Route Optimization:** Flexicab leverages real-time traffic data and historical ride patterns to suggest the most efficient routes for each trip, minimizing travel time and fuel consumption. This feature helps drivers maximize their earnings by completing more rides within a given timeframe.* **Fare Calculation & Management:** Flexicab automatically calculates fares based on distance, time, and other factors, ensuring transparency and accuracy for both drivers and passengers. It also integrates with popular payment gateways, allowing for seamless digital transactions and reducing the risk of cash handling.* **Customer Management:** Flexicab provides a platform for drivers to manage their customer base, track ride history, and build loyalty through personalized offers and promotions. This helps drivers attract repeat customers and increase their overall revenue.* **Fleet Management:** For Rickshaw owners with multiple vehicles, Flexicab offers a comprehensive fleet management system. This includes vehicle tracking, maintenance scheduling, driver performance monitoring, and real-time communication tools to ensure efficient operations and safety.* **Data Analytics & Insights:** Flexicab provides valuable insights into driver performance, customer behavior, and market trends. This data can be used to optimize pricing strategies, identify high-demand areas, and make informed business decisions.**Benefits:*** **Increased Earnings:** By optimizing routes, managing fares effectively, and attracting more customers, Flexicab helps drivers significantly increase their earnings.* **Improved Efficiency:** Flexicab streamlines operations, reducing wasted time and effort, allowing drivers to focus on providing excellent service.* **Enhanced Safety:** The platform's features, such as vehicle tracking and driver performance monitoring, contribute to a safer environment for both drivers and passengers.* **Digital Transformation:** Flexicab empowers Rickshaw drivers and owners to embrace digital technology, making their businesses more modern and competitive.**Target Audience:*** Rickshaw drivers* Rickshaw owners* Transportation companies operating Rickshaws* City governments and transportation authorities looking to improve urban mobility and support sustainable transportation options.**Flexicab is more than just a platform; it's a comprehensive solution that empowers Rickshaw drivers and owners to thrive in the modern urban landscape.
Query: How does Flexicab help Rickshaw drivers increase their earnings?
Instructions:You are a helpful and informative chatbot designed to answer questions about Flexicab, a SaaS platform for Rickshaw drivers and owners. Use the provided context to answer the user's query in a comprehensive and engaging manner. **Response Format:*** Begin your response with a clear and concise answer to the user's question.* Provide additional details and explanations to support your answer, drawing from the provided context.* Use bullet points or numbered lists to organize information and make it easier to read.* If applicable, include specific examples or scenarios to illustrate your points.* End your response with a friendly and helpful tone, encouraging further interaction.**
"""



model = TextGenerationModel.from_pretrained("text-bison")
parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 40
}
response = model.predict(prompt, **parameters)
print(response.text)

 Flexicab helps Rickshaw drivers increase their earnings in several ways:

**1. Route Optimization:**
- Flexicab uses real-time traffic data and historical ride patterns to suggest the most efficient routes for each trip.
- This reduces travel time and fuel consumption, allowing drivers to complete more rides within a given timeframe.
- By optimizing their routes, drivers can increase the number of rides they complete each day, leading to higher earnings.

**2. Fare Calculation & Management:**
- Flexicab automatically calculates fares based on distance, time, and other factors, ensuring transparency and accuracy for both drivers and passengers.
- This eliminates disputes over fares and ensures that drivers receive fair compensation for their services.
- Additionally, Flexicab integrates with popular payment gateways, allowing for seamless digital transactions and reducing the risk of cash handling.

**3. Customer Management:**
- Flexicab provides a platform for drivers to manage their 

# Evaluating Responses with Llumo

### Evaluate the OpenAI Response:
* We evaluate the response generated by the OpenAI API using the Llumo evaluation function.
* We call the evaluate_with_llumo function with the example prompt and the openai_output.
* We check if the evaluation was successful:
  * If successful, we print the Llumo evaluation results.
  * If the evaluation fails, we print an error message and indicate that the original prompt can be used if evaluation fails.

In [33]:
print("Evaluating Responses:")
eval_result, success = evaluate_with_llumo(prompt, response.text)

if success:
    print("Llumo Evaluation Results:")
    for analytic, score in eval_result.items():
        print(f"{analytic}: {score}")
else:
    print("Evaluation with Llumo failed.")


Evaluating Responses:
API Response: {'data': {'data': '{"analyticsScore": {"professional_tone": 80, "overallScore": 80}, "reasoning": {"professional_tone": ["The output uses formal language and maintains a respectful tone throughout.", "The response is well-organized and easy to read, using bullet points and numbered lists to present information clearly.", "The chatbot demonstrates expertise by accurately referencing the provided context and providing relevant information.", "The response is helpful and informative, addressing the user\'s query in a comprehensive manner."], "overallScore": "Calculated based on individual metric scores"}, "inputTokenCount": 631, "outputTokenCount": 306, "totalToken": 937, "cost": 0.00046849999999999995, "isRepeat": false, "isError": true, "isRefusal": false}', 'statusCode': 200, 'message': 'SUCCESS'}}
Llumo Evaluation Results:
data: {"analyticsScore": {"professional_tone": 80, "overallScore": 80}, "reasoning": {"professional_tone": ["The output uses for

## Testing on Diverse Outputs
Let us an example with two outputs in which one with good score and one with low score. For example, we are using Professionalism as definition here.

In [34]:
prompt = """Context: Flexicab: The Smart Rickshaw Management PlatformFlexicab is a revolutionary SaaS platform designed specifically for Rickshaw drivers and owners in bustling urban environments. It addresses the unique challenges faced by this vital mode of transportation, empowering them with modern tools to optimize their operations and increase their earnings.**Key Features:*** **Route Optimization:** Flexicab leverages real-time traffic data and historical ride patterns to suggest the most efficient routes for each trip, minimizing travel time and fuel consumption. This feature helps drivers maximize their earnings by completing more rides within a given timeframe.* **Fare Calculation & Management:** Flexicab automatically calculates fares based on distance, time, and other factors, ensuring transparency and accuracy for both drivers and passengers. It also integrates with popular payment gateways, allowing for seamless digital transactions and reducing the risk of cash handling.* **Customer Management:** Flexicab provides a platform for drivers to manage their customer base, track ride history, and build loyalty through personalized offers and promotions. This helps drivers attract repeat customers and increase their overall revenue.* **Fleet Management:** For Rickshaw owners with multiple vehicles, Flexicab offers a comprehensive fleet management system. This includes vehicle tracking, maintenance scheduling, driver performance monitoring, and real-time communication tools to ensure efficient operations and safety.* **Data Analytics & Insights:** Flexicab provides valuable insights into driver performance, customer behavior, and market trends. This data can be used to optimize pricing strategies, identify high-demand areas, and make informed business decisions.**Benefits:*** **Increased Earnings:** By optimizing routes, managing fares effectively, and attracting more customers, Flexicab helps drivers significantly increase their earnings.* **Improved Efficiency:** Flexicab streamlines operations, reducing wasted time and effort, allowing drivers to focus on providing excellent service.* **Enhanced Safety:** The platform's features, such as vehicle tracking and driver performance monitoring, contribute to a safer environment for both drivers and passengers.* **Digital Transformation:** Flexicab empowers Rickshaw drivers and owners to embrace digital technology, making their businesses more modern and competitive.**Target Audience:*** Rickshaw drivers* Rickshaw owners* Transportation companies operating Rickshaws* City governments and transportation authorities looking to improve urban mobility and support sustainable transportation options.**Flexicab is more than just a platform; it's a comprehensive solution that empowers Rickshaw drivers and owners to thrive in the modern urban landscape.
Query: How does Flexicab help Rickshaw drivers increase their earnings?
Instructions:You are a helpful and informative chatbot designed to answer questions about Flexicab, a SaaS platform for Rickshaw drivers and owners. Use the provided context to answer the user's query in a comprehensive and engaging manner. **Response Format:*** Begin your response with a clear and concise answer to the user's question.* Provide additional details and explanations to support your answer, drawing from the provided context.* Use bullet points or numbered lists to organize information and make it easier to read.* If applicable, include specific examples or scenarios to illustrate your points.* End your response with a friendly and helpful tone, encouraging further interaction.** """

good_output = """Flexicab helps Rickshaw drivers increase their earnings by optimizing their routes, managing fares effectively, and attracting more customers. Here's how:

Route Optimization: Flexicab uses real-time traffic data and historical ride patterns to suggest the most efficient routes. This minimizes travel time and fuel consumption, allowing drivers to complete more rides within a given timeframe, ultimately increasing their earnings.
Fare Calculation & Management: The platform calculates fares based on distance, time, and other factors, ensuring transparency and accuracy. This not only builds trust with passengers but also allows drivers to earn fair compensation for their services.
Customer Management: Flexicab enables drivers to manage their customer base and track ride history. By offering personalized promotions and maintaining strong customer relationships, drivers can attract repeat customers, boosting their revenue.
Fleet Management: For drivers managing multiple rickshaws, Flexicab provides tools for vehicle tracking, maintenance scheduling, and driver performance monitoring. These features help maintain efficiency and safety, contributing to a more profitable operation.
With Flexicab, Rickshaw drivers can enjoy increased earnings through smarter, more efficient operations. If you have any more questions about Flexicab, feel free to ask! """


print("Evaluating the example with good score:")
llumo_result, success = evaluate_with_llumo(prompt, good_output)

if success:
    print("Llumo Evaluation Results:")
    for analytic, score in llumo_result.items():
        print(f"{analytic}: {score}")
else:
    print("Evaluation with Llumo failed.")


bad_output = """Flexicab makes Rickshaw drivers earn more. It has some features like fare calculation and route stuff. It helps them get more rides. It's good for them."""

print("Evaluating the example with low score:")

llumo_result, success = evaluate_with_llumo(prompt, bad_output)

if success:
    print("Llumo Evaluation Results:")
    for analytic, score in llumo_result.items():
        print(f"{analytic}: {score}")
else:
    print("Evaluation with Llumo failed.")


Evaluating the example with good score:
API Response: {'data': {'data': '{"analyticsScore": {"professional_tone": 80, "overallScore": 80}, "reasoning": {"professional_tone": ["The output uses formal language and maintains a respectful tone throughout.", "The response is clear, concise, and well-organized, demonstrating professionalism.", "The chatbot provides a helpful and engaging tone, encouraging further interaction, which aligns with professional communication."], "overallScore": "Calculated based on individual metric scores"}, "inputTokenCount": 631, "outputTokenCount": 238, "totalToken": 869, "cost": 0.0004345, "isRepeat": false, "isError": false, "isRefusal": true}', 'statusCode': 200, 'message': 'SUCCESS'}}
Llumo Evaluation Results:
data: {"analyticsScore": {"professional_tone": 80, "overallScore": 80}, "reasoning": {"professional_tone": ["The output uses formal language and maintains a respectful tone throughout.", "The response is clear, concise, and well-organized, demonstra